In [4]:
#0:最黑，255：最亮
import cv2
import numpy as np
#np.set_printoptions(threshold=np.inf)  #全部打印图片信息
from numpy import *
import matplotlib.pyplot as plt    
import operator 

def imgpreprocess():  #图片的前处理
    
    hsv=cv2.cvtColor(img,cv2.COLOR_BGR2HSV)
    #cv2.imshow('hsv空间图',hsv)
    #print(hsv)
    
    #读取灰度图
    img1=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    #cv2.imshow('灰度图',img1)
    #print(img1)

    #提取红色区域

    lower_blue=np.array([0,43,46])
    upper_blue=np.array([10,255,255])
    mask=cv2.inRange(hsv,lower_blue,upper_blue)

    lower_red=np.array([156,43,46])
    upper_red=np.array([180,255,255])
    mask1=cv2.inRange(hsv,lower_red,upper_red)
    
    mask2=cv2.add(mask,mask1)   #将2幅图融合一下,这个已经是二值化的图了。
    #cv2.imshow('提取整幅图的红色区域',mask2)
    #print(mask2)
    #print('提取整副图的红色区域',type(mask),mask2.shape)
    #cv2.imshow('mask',mask)


    #二值化
    #ret,binary=cv2.threshold(mask2,80,255,cv2.THRESH_BINARY)
    #cv2.imshow('blurred binary',binary)

    #模糊
    #blurred=cv2.blur(binary,(4,4))   #均值滤波
    #cv2.imshow('blurred',blurred)
    #"""
    #试试中值滤波
    blurred=cv2.medianBlur(mask2,7)
    #cv2.imshow('中值滤波之后的效果',blurred)
    #"""
    #使区域闭合无空隙
    kernelclose = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (7, 7)) #定义一个椭圆的核元素
  
    kernelopen=cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(7,7))
    #闭运算，就是先膨胀后腐蚀操作。
    closed = cv2.morphologyEx(blurred, cv2.MORPH_CLOSE, kernelclose) #闭合操作，把漏洞补上
    cv2.imshow('形态学操作，进行闭运算',closed)

    #腐蚀和膨胀
    '''
    腐蚀操作将会腐蚀图像中白色像素，以此来消除小斑点，
    而膨胀操作将使剩余的白色像素扩张并重新增长回去。
    '''

    erode=cv2.erode(closed,kernelopen,iterations=1)
    #cv2.imshow('腐蚀操作一次',erode)
    dilate=cv2.dilate(erode,kernelclose,iterations=2)  #iteration 指的是腐蚀次数，默认腐蚀一次，第二个参数，kernel，是多大的卷积核进行运算。
    #cv2.imshow('膨胀操作2次',dilate)

    #下面是对dilate闭操作的眼膜进行 填充空缺，好好研究下，下面有按位或和取反操作。
    copyimage=dilate.copy()    #先将眼膜copy出来
    h,w=dilate.shape[:2]       #读出眼膜的长、宽出来
    print(h,w)
    yanmo=np.zeros([h+2,w+2],np.uint8)    #定义一个yanmo模板，要比原图大2个像素位

    cv2.floodFill(copyimage,yanmo,None,(255),cv2.FLOODFILL_MASK_ONLY)  #用floodfill填充白色，注意这个他娘的把外围的黑色区域（0）填充为白色了，所以下面要取反
    
    #cv2.imshow("使用掩膜填充之后的图像",copyimage)
    zuizhong=dilate|(~copyimage)   #将填充后的图取反与原图做or运算，取反
    #cv2.imshow("填充之后,并取反与原图做或运算",zuizhong)
    return zuizhong,img1


def splitfrommid(img):  #resize ROI area into sample's size
    x,y=img.shape[:2]
    temp=img[:,:y//2]
    #print(temp.shape)
    temp=cv2.resize(temp,(32,32))
    #cv2.imshow('temp',temp)
    return temp



def processcontours():
    i=0
    res=img.copy()

    for c in contours:

        #轮廓转换为最小矩形
        rect=cv2.minAreaRect(c)
        #print(rect)
        #外接矩形的宽
        width=rect[1][0]
        #外接矩形高
        height=rect[1][1]
        #最小矩形框的面积
        area=width*height
        #长宽限制
        ratio=width/height
         #求解钜特征
        M=cv2.moments(c)
        objectarea=M['m00']
        #print('钜特征提取的面积%d'%(objectarea))

        if (ratio<=0.5 or ratio>=2)|(objectarea<2000):
            continue

        """
        #识别出来的轮廓面积
        contoursArea=cv2.contourArea(c)
        print('识别出来的轮廓面积%d'%(contoursArea))
        """

        #最小外接矩形转换为box
        box=np.int0(cv2.boxPoints(rect))  #这个就是最小矩形框的四个角点，有偏转度的，为了下面画图做的
        #print(box)

        if (objectarea/area)>0.9 and (objectarea/area)<1:
            print('该标示牌为矩形标示牌')
            cv2.drawContours(res, [box], 0, (255, 0, 0), 8)
        elif (objectarea/area)>0.7 and (objectarea/area)<0.85:
            print('该标示牌为圆形标示牌')
            cv2.drawContours(res, [box], 0, (0, 255, 0), 8)
  
            #找外接矩形的四个点
            x,y,w,h=cv2.boundingRect(c)
            #加上防错处理，确保裁剪区域无异常
            if w>0 and h>0:
                #裁剪矩形区域
                temp=img1[y:y+h,x:x+w]
                #cv2.imshow('signoriginal',temp)
                yy=y+h//4
                yyy=y+3*h//4
                xx=x+w//4
                xxx=x+3*w//4
                temp1=img1[yy:yyy,xx:xxx] #按照一半的大小截取出来内核
                caijian=splitfrommid(temp1)
                i=i+1
                ret,binary=cv2.threshold(temp1,80,255,cv2.THRESH_BINARY)
                #显示裁剪后的标志
                #cv2.imshow('sign'+str(i),temp1)



        elif (objectarea/area)>0.4 and (objectarea/area)<0.6:
            print("该标示牌为三角形标示牌")
            cv2.drawContours(res, [box], 0, (0, 0, 255), 8)
        else:
            print("没有检测到标示牌")  
    return res,caijian
    
def pca(dataMat, topNfeat):  
    global redEigVects  
    meanVals = np.mean(dataMat, axis=0)      
    meanRemoved = dataMat - meanVals #对应的减去每个维度的差值，得到新的矩阵 
    covMat = np.cov(meanRemoved, rowvar=0)  
    eigVals,eigVects = np.linalg.eig(mat(covMat))  
    eigValInd = np.argsort(eigVals)#sort, 从小到大排列
    eigValInd = eigValInd[:-(topNfeat+1):-1]  #把特征值的索引从大到小找出来  
    redEigVects = eigVects[:,eigValInd]   #重新把特征向量从大到小裁剪出来 
    #得到低维度数据  
    lowDDataMat = meanRemoved * redEigVects#降维过后的新的向量
    
    reconMat = (lowDDataMat * redEigVects.T) + meanVals  
    return lowDDataMat, reconMat  

def KNN(inX, dataSet, labels, k):  
    dataSetSize = dataSet.shape[0]  #大小为2500，训练集的大小，读出多少个样本，行数读取
   
    diffMat = tile(inX, (dataSetSize,1)) - dataSet #tile是将inx数组重复n次,把相对距离的x y 求出来  

    sqDiffMat = diffMat**2  #表示2次方
    sqDistances = sqDiffMat.sum(axis=1) #axis=0, 表示列。axis=1, 表示行(按照行向量相加）。  
    distances = sqDistances**0.5  #表示开根号
    sortedDistIndicies = distances.argsort()  #np排序，代表的是返回从小到大排序的下标  
    #print("计算出来待识别样本与特征库的距离：",sortedDistIndicies)
    classCount={}  
    
    #下面取欧式距离计算
    for i in range(k):  
        voteIlabel = labels[sortedDistIndicies[i]]  
        classCount[voteIlabel] = classCount.get(voteIlabel,0) + 1 
        print(classCount)
        # #get返回指定键的值，否则返回默认值  
    sortedClassCount = sorted(classCount.items(), key=operator.itemgetter(1), reverse=True)
    print(sortedClassCount)
    return sortedClassCount[0][0] 




if __name__ == "__main__":
    
    #定义一个全局特征转换变量　这个变量是在PCA中求出的  
    global redEigVects  
    
    #加载原图
    img=cv2.imread('testpic/9.jpg')
    h,w=img.shape[:2] 
    #print('原始图像的高与宽：',(h,w))
    #b,g,r=cv2.split(img)
    #print(g)
    
    #print(img)
    
    zuizhong,img1=imgpreprocess()

    #cv2.floodFill(copyimage,yanmo,None,(0, 100, 255), (100, 100, 50), (50, 50, 50), cv2.FLOODFILL_FIXED_RANGE) 

    # 查找轮廓
    image,contours, hierarchy=cv2.findContours(zuizhong.copy(), cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)#cv2.RETR_EXTERNAL：代表仅仅检测外轮廓

    #contour=np.array(contours)
    #print(len(contour))
    #np.savetxt('contours.txt',contour,fmt='%s')

    res,caijian=processcontours() #将图片roi区域裁剪出来，
    
    trainNum=109
    dimensionR=50 #初始化要降的维度
    trainingMatorigin=np.zeros((trainNum,32*32))
    #print(trainingMatorigin.shape) [109,1024]

    #初始化标签
    trainingLabel=[]

    a=np.ones((14,1),dtype=np.int) #生成14行一列的1数组"1",并且指明是整形的，不然默认float类型的
    b=np.ones((12,1),dtype=np.int)*2
    c=np.ones((11,1),dtype=np.int)*3
    d=np.ones((13,1),dtype=np.int)*4
    e=np.ones((12,1),dtype=np.int)*5
    f=np.ones((13,1),dtype=np.int)*6
    g=np.ones((16,1),dtype=np.int)*7
    h=np.ones((10,1),dtype=np.int)*8
    j=np.ones((8,1),dtype=np.int)*9


    for i in range(trainNum):
        img=cv2.imread('MNIST_data/'+str(i+1)+ '.png',0)
        ret,binary=cv2.threshold(img,80,255,cv2.THRESH_BINARY)
        binary=np.reshape(binary,1*1024) #将二值化的图片转化成1行，1024列的数组,记住一定是乘法
        #print(binary.shape)
        trainingMatorigin[i]=binary
        #下面是将label 赋值

        if i<14:
            trainingLabel.append(a[i][0])
        if 14<=i<26:
            trainingLabel.append(b[i-15][0])
        if 26<=i<37:
            trainingLabel.append(c[i-27][0])
        if 37<=i<50:
            trainingLabel.append(d[i-38][0])
        if 50<=i<62:
            trainingLabel.append(e[i-51][0])
        if 62<=i<75:
            trainingLabel.append(f[i-63][0])
        if 75<=i<91:
            trainingLabel.append(g[i-76][0])
        if 91<=i<101:
            trainingLabel.append(h[i-92][0])
        if i>=101:
            trainingLabel.append(j[i-102][0])

    print("标签有%d个"%(len(trainingLabel)))

    #下面开始降维
    trainingMat,reconMat=pca(trainingMatorigin,dimensionR)
    print("打印出来降成的维度为：",trainingMat[1].shape)  #成功，打印出来的降维过后的矩阵为（1,40），也就是从1024维度降低到了40维度


    #下面进行测试集的读取测试
    #测试数据
    testNum=1
    #错误计数
    errcount=0

    #待检测数据的真实标签
    biaoqian=np.array([3],dtype=np.int)

    for i in range(testNum):
        img=caijian
        ret,binar=cv2.threshold(img,80,255,cv2.THRESH_BINARY)
        binar=np.reshape(binar,1*1024) #将二值化的图片转化成1行，1024列的数组
        meanVals=mean(binar,axis=0)
        meanRemoved=binar-meanVals
        testingMat=meanRemoved*redEigVects
        #print('切割出来的标示牌降维后的特征向量',testingMat)
        #读取待测数字的真实标签
        zhenb=biaoqian[i]
        print(zhenb)
        #print(type(zhenb))  #int64 的numpy
        print("真实的数字是多少：",zhenb)
        classifierResult=KNN(testingMat.getA(),trainingMat.getA(),trainingLabel,3)
        print('最邻近分类出来的标签%d'%(classifierResult))
        #print(type(classifierResult)) #麻痹的，数据类型要对齐，否则全是错的

        if classifierResult!=zhenb:
            errcount=errcount+1
        print('错了几个了',errcount)


    print('错误率是：',float(errcount)/testNum) 
        #显示画了标志的原图       
    #cv2.imshow('img',res)
   
    cv2.waitKey(0)
    cv2.destroyAllWindows()


800 1200
该标示牌为圆形标示牌
标签有109个
打印出来降成的维度为： (1, 50)
3
真实的数字是多少： 3
{1: 1}
{1: 2}
{1: 3}
[(1, 3)]
最邻近分类出来的标签1
错了几个了 1
错误率是： 1.0
